In [101]:
import advent
data: list[tuple[str, str, str]] = advent.get_lines(18, 'txt', lambda s: s.split(" "))

def tadd(a: tuple[int, int], b: tuple[int, int]):
    return a[0] + b[0], a[1] + b[1]

In [102]:
dirmap = {'R': (0, 1), 'L': (0, -1), 'U': (-1, 0), 'D': (1, 0)}

def get_visited(data: list[tuple[str, str, str]]):
    visited: set[tuple[int, int]] = set([(0, 0)])
    current = (0, 0)
    for line in data:
        dir, size, _ = line
        size = int(size)
        while size > 0:
            size -= 1
            current = tadd(current, dirmap[dir])
            visited.add(current)
    return visited

def is_inside(row: int, col: int, visited: set[tuple[int, int]], minrow: int):
    # Approach: travel up (until row=-1), if intersections is even, its out
    # if intersections is odd, its inside
    # Intersection is defined as: (row-n, col) and (row-n, col+1) are visited
    if (row, col) in visited: return True
    inside = False
    while row >= minrow:
        row -= 1
        if (row, col) in visited and (row, col+1) in visited:
            inside = not inside
    return inside

visited = get_visited(data)
maxrow, maxcol = max(s[0] for s in visited), max(s[1] for s in visited)
minrow, mincol = min(s[0] for s in visited), min(s[1] for s in visited)
result = 0
for row in range(minrow, maxrow+1):
    for col in range(mincol, maxcol+1):
        if is_inside(row, col, visited, minrow): result += 1
print(result)

45159


In [103]:
# Part 2.
# I I knew (from day 10) that I could treat the vertices
# as a polygon and calculate the interior. But I decided not to use shapely
# and instead learn how area calculation works
# So I found: https://stackoverflow.com/a/451482/1615209

# logic for determining the 'interior points': (done without help)
# let's say area is 100, meaning we have 100 1x1 blocks,
# meaning we have 400 corners. 
# That does not mean our interior points are 100, since the outside points
# got undercounted: e.g. a corner point only adds 1 to the 400, an edge only 2
# So to make all interior points count as 4 (so we can divide by 4 later)
# We add 3 for every corner edge, and 2 for every non-corner edge.
# But its easier to just add 2 for every edge, then another 1 for every corner

def interior(area: int, perimeter: int, corners: int) -> float:
    # NOTE: I later found out this only works for convex polygons
    return ((area*4) + (perimeter*2) + (corners)) / 4

assert interior(9, 12, 4) == 16

def decode(line: str) -> tuple[int, str]:
    # The reason why return type is weird to keep it the same type as input data
    edge_length, dir = line[2:7], line[7]
    dirmap = ['R', 'D', 'L', 'U']
    return int(f"0x{edge_length}", 16), dirmap[int(dir)]

assert decode("(#000012)") == (1, 'L')

def get_visited_corners(data: list[tuple[int, str]]):
    visited: list[tuple[int, int]] = list([(0, 0)])
    current = (0, 0)
    for line in data:
        size, dir = line
        current = tadd(current, (size * dirmap[dir][0], size * dirmap[dir][1]))
        visited.append(current)
    return visited


In [104]:
# Now all that's left is to calculate corners, perimeter, area
# corners is easy: assuming that the data doesnt contain 'tricks' like
# R followed by another R, then corners is simply len(data)
corners = len(data)

# perimeter is sum of length of edges
perimeter = sum(int(decode(line[2])[0]) for line in data)

# Calculate vertices as in part 1
visited = get_visited_corners([decode(line[2]) for line in data])

# From stackoverflow
def get_area(p: list[tuple[int, int]]):
    segments = zip(p[:-1], p[1:])
    return 0.5 * abs(sum(x0*y1 - x1*y0
                         for ((x0, y0), (x1, y1)) in segments))

area = get_area(visited)

print(corners, perimeter, area)
print(interior(int(area), perimeter, corners))

# It doesnt work because I made a logical error:
# The corners dont always add 1 point. The convex corners do
# The concave corners actually add 3...

664 173561416 134549208019004.0
134549294799878.0


In [106]:
def count_convex(turns: list[str]):
    # input: list like ['R', 'D', 'L'], etc.
    # L must always be followed by D or U, etcetera.
    # Returns: number of convex corners

    # This works when clockwise. When anticlockwise, the opposite is true
    # The polygon being clockwise is essentialy an assumption, boiling down to:
    # The 'inside' is on our right side as we walk the edges
    convex = {'R': 'D', 'L': 'U', 'U': 'R', 'D': 'L'}
    return sum(convex[a] == b for a, b in zip(turns, turns[1:] + [turns[0]]))

def interior_fixed(area: float, perimeter: int, corners: int, convex: int) -> float:
    concave = corners - convex
    return ((area*4) + (perimeter*2) + (corners) - (concave*2)) / 4

convex = count_convex([decode(line[2])[1] for line in data])
print(int(interior_fixed(area, perimeter, corners, convex)))

134549294799713
